# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [1]:
# This data represent 17 marketing campaigns

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import time
from sklearn import metrics
from sklearn.dummy import DummyClassifier

In [3]:
# Read and create dataframe for bank products marketing data
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [4]:
# Look at the snapshot of the loded data in dataframe
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [5]:
# look at the data information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [6]:
# Note the shape of the dataframe
df.shape

(41188, 21)

In [7]:
# Note the description of the dataframe
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [8]:
# Check if any null values
df.isnull().sum() 

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [9]:
# check if any missing values
df.isna().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [10]:
# Check if there are 0s as value
df.eq(0).sum() /41188 * 100

age                0.000000
job                0.000000
marital            0.000000
education          0.000000
default            0.000000
housing            0.000000
loan               0.000000
contact            0.000000
month              0.000000
day_of_week        0.000000
duration           0.009712
campaign           0.000000
pdays              0.036418
previous          86.343110
poutcome           0.000000
emp.var.rate       0.000000
cons.price.idx     0.000000
cons.conf.idx      0.000000
euribor3m          0.000000
nr.employed        0.000000
y                  0.000000
dtype: float64

In [11]:
# Note the unique value counts for each coloumn
df.nunique()

age                 78
job                 12
marital              4
education            8
default              3
housing              3
loan                 3
contact              2
month               10
day_of_week          5
duration          1544
campaign            42
pdays               27
previous             8
poutcome             3
emp.var.rate        10
cons.price.idx      26
cons.conf.idx       26
euribor3m          316
nr.employed         11
y                    2
dtype: int64

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

### Business Objective of the task: 
##### For the given marketing bank products data, create K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines classifiers.  Measure and compare the performance of each classifier.

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [13]:
# Drop the column duration based on the earlier data details provided
cleaned_df = df.drop(columns=['duration'],inplace=False)

In [14]:
# Check the shape of the dataframe
cleaned_df.shape

(41188, 20)

In [15]:
# Set X and y data
X = cleaned_df.drop('y',axis = 1)
y = cleaned_df['y']

In [16]:
# Check the shape of the X data
X.shape

(41188, 19)

In [17]:
# Perform transformation.  Use one hot coding for cateogrical variables
# Gather categorical variables from the dataframe to pass to one hot encoding 
categorical_columns = X.select_dtypes(include=['object']).columns
categorical_columns_df = cleaned_df[categorical_columns]
print(categorical_columns_df.shape)
print(categorical_columns)
oneHotEncoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_data = oneHotEncoder.fit_transform(categorical_columns_df)

(41188, 10)
Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'day_of_week', 'poutcome'],
      dtype='object')


In [18]:
# Create dataframe with encoded features
encoded_categorical_df = pd.DataFrame(encoded_data, columns=oneHotEncoder.get_feature_names_out())

print(encoded_categorical_df.shape)
encoded_categorical_df.head()

(41188, 53)


,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [19]:
# scale numerical data and then concat numerical and encoded categorical features.
numerical_columns = X.select_dtypes(include=['float64','int64']).columns
numerical_columns_df = cleaned_df[numerical_columns]
print(numerical_columns_df.shape)
print(numerical_columns)

(41188, 9)
Index(['age', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'],
      dtype='object')


In [20]:
# use Standard Scalar to scale the numeric data
scaler = StandardScaler()
scaled_numerical_data = scaler.fit_transform(numerical_columns_df)

In [21]:
# create scaled_numerical_data frame
scaled_dataframe = pd.DataFrame(scaled_numerical_data, columns=numerical_columns_df.columns)

In [22]:
# create dataframe that contains both numerical and categorical transformed features
transformed_features_df = pd.concat([scaled_dataframe, encoded_categorical_df], axis=1)
print(transformed_features_df)

            age  campaign     pdays  previous  emp.var.rate  cons.price.idx  \
0      1.533034 -0.565922  0.195414 -0.349494      0.648092        0.722722   
1      1.628993 -0.565922  0.195414 -0.349494      0.648092        0.722722   
2     -0.290186 -0.565922  0.195414 -0.349494      0.648092        0.722722   
3     -0.002309 -0.565922  0.195414 -0.349494      0.648092        0.722722   
4      1.533034 -0.565922  0.195414 -0.349494      0.648092        0.722722   
...         ...       ...       ...       ...           ...             ...   
41183  3.164336 -0.565922  0.195414 -0.349494     -0.752343        2.058168   
41184  0.573445 -0.565922  0.195414 -0.349494     -0.752343        2.058168   
41185  1.533034 -0.204909  0.195414 -0.349494     -0.752343        2.058168   
41186  0.381527 -0.565922  0.195414 -0.349494     -0.752343        2.058168   
41187  3.260295  0.156105  0.195414  1.671136     -0.752343        2.058168   

       cons.conf.idx  euribor3m  nr.employed  job_a

In [23]:
# Check the shape of the transformed features dataframe
transformed_features_df.shape

(41188, 62)

In [24]:
# Assign transfomed features dataframe as X
X = transformed_features_df
X.shape

(41188, 62)

In [25]:
# Verify the transformed features information
transformed_features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 62 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   age                            41188 non-null  float64
 1   campaign                       41188 non-null  float64
 2   pdays                          41188 non-null  float64
 3   previous                       41188 non-null  float64
 4   emp.var.rate                   41188 non-null  float64
 5   cons.price.idx                 41188 non-null  float64
 6   cons.conf.idx                  41188 non-null  float64
 7   euribor3m                      41188 non-null  float64
 8   nr.employed                    41188 non-null  float64
 9   job_admin.                     41188 non-null  float64
 10  job_blue-collar                41188 non-null  float64
 11  job_entrepreneur               41188 non-null  float64
 12  job_housemaid                  41188 non-null 

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(type(X_train), type(y_train))

(32950, 62)
(8238, 62)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [27]:
dummy_clf = DummyClassifier().fit(X_train, y_train)
baseline_score = dummy_clf.score(X_test, y_test)
print(baseline_score)

0.8865015780529255


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [28]:
LR_basic = LogisticRegression(random_state=42)
LR_basic.fit(X_train,y_train)
LR_basic_acc = LR_basic.score(X_test, y_test)

### Problem 9: Score the Model

What is the accuracy of your model?

In [29]:
print(LR_basic_acc)

0.8971837824714737


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [30]:
# Model Comparisons Result store
results  = {
    "Model": [],
    "Train Time": [],
    "Train Accuracy": [],
    "Test Accuracy": []
}

In [31]:
# Models to run
models = {
    'Logistic Regression': LogisticRegression(),
    'kNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'SVM': SVC()
}

In [32]:
# fit for each model and collect the results for train time, test and train accuracy
for model_name, model in models.items():
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_accuracy = metrics.accuracy_score(y_train, y_train_pred)
    test_accuracy = metrics.accuracy_score(y_test, y_test_pred)

    results['Model'].append(model_name)
    results["Train Time"].append(train_time)
    results['Test Accuracy'].append(test_accuracy)
    results['Train Accuracy'].append(train_accuracy)

In [33]:
# Store results in dataframe
results_df = pd.DataFrame(results)

In [34]:
#  Display the results
results_df

,Model,Train Time,Train Accuracy,Test Accuracy
0,Logistic Regression,0.092844,0.901153,0.897184
1,kNN,0.018561,0.914234,0.888080
2,Decision Tree,0.157129,0.995357,0.838067
3,SVM,20.663825,0.904795,0.896941


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [35]:
# Improved Model Comparisons Result store
tuned_results  = {
    "Model": [],
    "Train Time": [],
    "Train Accuracy": [],
    "Test Accuracy": []
}

In [36]:
# 1st run Run grid search CV for kNN model and fine the best k
kNN_model = KNeighborsClassifier()
params = {'n_neighbors': range(1,21)}
grid_search_knn = GridSearchCV(estimator=kNN_model,
                              param_grid=params)
grid_search_knn.fit(X_train,y_train)
best_k = grid_search_knn.best_params_['n_neighbors']
print(best_k)

20


In [37]:
# use the best k from above and run the model and print the results
kNN_best_model = KNeighborsClassifier(n_neighbors=20)
start_time = time.time()
kNN_best_model.fit(X_train,y_train)
train_time = time.time() - start_time

y_train_pred = kNN_best_model.predict(X_train)
y_test_pred = kNN_best_model.predict(X_test)

train_accuracy = metrics.accuracy_score(y_train, y_train_pred)
test_accuracy = metrics.accuracy_score(y_test, y_test_pred)

tuned_results['Model'].append('kNN_with_20_neighbors')
tuned_results["Train Time"].append(train_time)
tuned_results['Test Accuracy'].append(test_accuracy)
tuned_results['Train Accuracy'].append(train_accuracy)

print(train_time)
print(train_accuracy)
print(test_accuracy)


0.01938605308532715
0.9035811836115326
0.8968196164117505


In [38]:
# 2nd Run of grid search CV for kNN model with n_neighbors and weights param tuning

param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],  
    'weights': ['uniform', 'distance']
}
kNN_model = KNeighborsClassifier()
grid_search_knn = GridSearchCV(estimator=kNN_model,
                              param_grid=param_grid)
grid_search_knn.fit(X_train,y_train)
best_k = grid_search_knn.best_params_['n_neighbors']
print(best_k)
print("Best parameters:", grid_search_knn.best_params_)
print("Best score (accuracy):", grid_search_knn.best_score_) 

15
Best parameters: {'n_neighbors': 15, 'weights': 'uniform'}
Best score (accuracy): 0.8979666160849773


In [39]:
# run the model with the above parameters, collect and print the results
kNN_best_model = KNeighborsClassifier(n_neighbors=15,weights='uniform')
start_time = time.time()
kNN_best_model.fit(X_train,y_train)
train_time = time.time() - start_time

y_train_pred = kNN_best_model.predict(X_train)
y_test_pred = kNN_best_model.predict(X_test)

train_accuracy = metrics.accuracy_score(y_train, y_train_pred)
test_accuracy = metrics.accuracy_score(y_test, y_test_pred)

tuned_results['Model'].append('kNN_with_15_neighbors_uniform')
tuned_results["Train Time"].append(train_time)
tuned_results['Test Accuracy'].append(test_accuracy)
tuned_results['Train Accuracy'].append(train_accuracy)

print(train_time)
print(train_accuracy)
print(test_accuracy)

0.018793106079101562
0.9047040971168437
0.8937848992473901


In [40]:
# Tune Decision Tree using GridSearchCV, collect and print the results
params = {'min_impurity_decrease': [0.01, 0.02, 0.03, 0.05],
         'max_depth': [2, 5, 10],
         'min_samples_split': [0.1, 0.2, 0.05]}
grid = GridSearchCV(DecisionTreeClassifier(random_state = 42), param_grid=params).fit(X_train, y_train)
grid_train_acc = grid.score(X_train, y_train)
grid_test_acc = grid.score(X_test, y_test)
best_params = grid.best_params_
print(f'Training Accuracy: {grid_train_acc: .2f}')
print(f'Test Accuracy: {grid_test_acc: .2f}')
print(f'Best parameters of tree: {best_params}')

Training Accuracy:  0.89
Test Accuracy:  0.89
Best parameters of tree: {'max_depth': 2, 'min_impurity_decrease': 0.01, 'min_samples_split': 0.1}


In [41]:
# Run the model with the above params
decisonTree_best_model = DecisionTreeClassifier(random_state = 42,min_impurity_decrease=0.01, max_depth=2, min_samples_split=0.1)
start_time = time.time()
decisonTree_best_model.fit(X_train,y_train)
train_time = time.time() - start_time

y_train_pred = decisonTree_best_model.predict(X_train)
y_test_pred = decisonTree_best_model.predict(X_test)

train_accuracy = metrics.accuracy_score(y_train, y_train_pred)
test_accuracy = metrics.accuracy_score(y_test, y_test_pred)

tuned_results['Model'].append('DecisionTree_multiple_params')
tuned_results["Train Time"].append(train_time)
tuned_results['Test Accuracy'].append(test_accuracy)
tuned_results['Train Accuracy'].append(train_accuracy)

print(train_time)
print(train_accuracy)
print(test_accuracy)

0.035156965255737305
0.887556904400607
0.8865015780529255


In [42]:
# create tuned results dataframe and display the results
tuned_results_df = pd.DataFrame(tuned_results)
tuned_results_df

,Model,Train Time,Train Accuracy,Test Accuracy
0,kNN_with_20_neighbors,0.019386,0.903581,0.896820
1,kNN_with_15_neighbors_uniform,0.018793,0.904704,0.893785
2,DecisionTree_multiple_params,0.035157,0.887557,0.886502
